<a href="https://colab.research.google.com/github/emily2925/machine_learning_algorithm/blob/main/Clustering%E5%AD%B8%E7%BF%92%E7%AD%86%E8%A8%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clustering演算法概要**
**軟分群：不一定會所有點都分群(待研究)**

**硬分群：所有點都會分群**



*   切割式分群：K系列
*   階層式分群：Hierarchy系列，相較於純分群可以看出兩倆關係和形成群集的順序，例如基因分群


>**使用場景**
* 純數值變數：KMeans, Hierarchy
* 純類別變數：KModes, Gower matrix + Hierarchy
* 數值＋類別變數：KPrototypes, Gower matrix + Hierarchy














**此筆記整理Kmeans,Kmodes,KPrototypes,Hierarchy之原理、使用場合、限制、前置作業、實作程式和結果解釋**<br>
*   Kmeans原理主要參考：https://www.youtube.com/watch?v=4b5d3muPQmA&ab_channel=StatQuestwithJoshStarmer
*   KModes原理主要參考：https://www.youtube.com/watch?v=b39_vipRkUo&ab_channel=AysanFernandes
*   KModes參考程式：https://www.analyticsvidhya.com/blog/2021/06/kmodes-clustering-algorithm-for-categorical-data/
*   KPrototypes原理主要參考：https://www.hindawi.com/journals/mpe/2020/5143797/
*   KMeans、KModes、KPrototype計算方式比較：https://www.jianshu.com/p/c9dcc52b85d4
*   KModes和KPrototype都可以用kmodes套件取用：https://pypi.org/project/kmodes/
*   Hierarchy原理主要參考：https://www.youtube.com/watch?v=7xHsRkOdVwo&ab_channel=StatQuestwithJoshStarmer
*   Hierarchy如用於mixed data要先用gower distance(1)：https://towardsdatascience.com/clustering-on-numerical-and-categorical-features-6e0ebcf1cbad
*   Hierarchy如用於mixed data要先用gower distance(2)：https://www.thinkdatascience.com/post/2019-12-16-introducing-python-package-gower/
















---



**Kmeans**

**1.原理** <br>
a. 先決定要k個群，n次嘗試(即n_init，預設10)，並隨意選定k個樣本點作為centroids<br>
b. 計算其他樣本點和這些centroids的距離(歐幾里德，平方加總開根號，variation, MSE)，和誰最接近則被分入該群<br>
c. 分完後將目前k個群集的平均值點作為k個新的centroids，並重複b和c步驟，直到cluster沒有變化<br>
d. 完成第一次的嘗試，並將各樣本點和最終該群的centroids之間的距離加總作為評估值
e. 重複a到d，但改變起始樣本點，直到做了共n次嘗試，將n次中，variation最小的嘗試作為k群n次的評估值

**2.使用場合** <br>
對於一群不明特質的群體希望進行分類<br>
a.希望對一群客戶精準行銷：用消費習慣、使用裝置分群<br>
b.金融詐騙樣態分群<br>
c.圖像<br>
d.文本<br>

**3.限制** <br>
a. 不能是類別變數或dummy variable，因其數值大小無意義，故數值距離沒有意義=>可用KModes取代，如為混合資料則可用KPrototypes<br>
b. 可能被離群值影響結果=>檢查、剔除看看結果差異<br>
c. 有些資料分佈分群結果不佳，特殊形狀或uniform=>可看各群各值均值或改用hierarchy<br>

**4.前置作業** <br>
a. (必做)Normalization：因為各feature的值大小可能因單位或其他因素差很多，為了不讓數值變異本就超大的feature被重點呈現，需要標準化，可除以std，讓標準差=1<br>
b. (必做)決定Seed：隨機選取初始值的方式一樣，可重現結果<br>
c. (必做)決定k值：是先跑出各k值的variation加總結果，找出elbow point，即variation下降最多的點<br>
b. (必做)檢查離群值<br>
d. (Optional)Feature Selection：先評估一下要放入的feature或用其他演算法輔助<br>






In [ ]:
# 套件
from numpy import random
from scipy.cluster.vq import whiten
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# 標準化 
df['scaled_x'] = whiten(df['x'])
df['scaled_y'] = whiten(df['y'])
df_dum_detail = df[['scaled_x', 'scaled_y']]

# 定好seed
random.seed(12)

In [ ]:
# 篩選k值：偵測現有下，k值應多少為最佳
sse = []
for k in range(1, 8):
  print(k)
  kmeans = KMeans(n_clusters=k)
  kmeans.fit(df_dum_detail)
  sse.append(kmeans.inertia_)

# k值篩選：圖像化篩選結果，SSE肘點為選擇點
plt.style.use("fivethirtyeight")
plt.plot(range(1, 8), sse)
plt.xticks(range(1, 8))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
# 分組
kmeans = KMeans(n_clusters=5)
labels = kmeans.fit_predict(df_dum_detail)
# 將分組資料 (分類標籤) 併入原資料
lb = pd.DataFrame(labels, columns=['labels'])
df_result = pd.concat((lb, df_dum_detail), axis=1)

# 商品類別分類的結果-每組數量
print(df_result.groupby("labels").count())

# 商品類別分類的結果-平均值
print(df_result.groupby("labels").mean())

# 圖像化結果
df_result.groupby('labels')[scaled_features].mean().plot(legend=True, kind='bar')
plt.show()

**KModes**

**整個過程、原理和KMeans幾乎一樣，只是算距離的方式有些不同，是用海明威距離，且不需事前先Normalization，以下說明計算距離的概念**<br>

a. 隨機選3點(假設k=3)作為Centroids<br>
b. 將每個點去和Centroids比較，例如先和C1比，比較時去看每個類別只要有不同即+1，則和C1的距離及加總，然後算C2距離、C3距離<br>
c. 每個點會被分類到和C距離最小的類別<br>
d. 接著重選新Centroids，選擇方式是把該Cluster所有點的每個feature值攤開看，每個feature都選目前最多點有的值，即產生Centroids<br>
e. 一直做到Cluster不變動<br>

In [ ]:
# 套件
from numpy import random
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# 定好seed
random.seed(12)

In [ ]:
# 篩選k值
cost = []
K = range(1,5)
for num_clusters in list(K):
    kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
    kmode.fit_predict(df_dum_detail)
    cost.append(kmode.cost_)

plt.plot(K, cost, 'bx-')
plt.xlabel('No. of clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# Building the model with clusters
kmode = KModes(n_clusters=2, init = "random", n_init = 5, verbose=1)
clusters = kmode.fit_predict(df_dum_detail)
df_dum_detail.insert(0, "Cluster", clusters, True)

In [ ]:
df_dum_detail.groupby('Cluster')['is_cart'].count()
df_dum_detail.groupby(['Cluster', 'is_cart']).count()

**KPrototypes**

**整個過程、原理和KMeans幾乎一樣，算距離的方式為數值變數用歐幾里德距離算，然後類別變數用KModes的方式算**<br>

In [ ]:
# 套件
from numpy import random
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# 定好seed
random.seed(12)

In [ ]:
# 篩選k值
cost = []
K = range(1,5)
for num_clusters in list(K):
    kprototypes = KPrototypes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
    kprototypes.fit_predict(df_dum_detail)
    cost.append(kprototypes.cost_)

plt.plot(K, cost, 'bx-')
plt.xlabel('No. of clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# Building the model with clusters
kprototypes = KPrototypes(n_clusters=2, init = "random", n_init = 5, verbose=1)
clusters = kprototypes.fit_predict(df_dum_detail)
df_dum_detail.insert(0, "Cluster", clusters, True)

In [ ]:
df_dum_detail.groupby('Cluster')['is_cart'].count()
df_dum_detail.groupby(['Cluster', 'is_cart']).count()

**Hierarchy**

**1.原理** <br>
a. 兩兩比較樣本點，用歐幾里得距離(多種選擇)，距離最近的併成一cluster<br>
b. 一樣兩兩比較，cluster和cluster間的距離比較方式也有多種，例如比較最近的點或最遠的點<br>
c. 最終併成一個群，但整個過程和每個群行成的距離、順序都可繪出<br>

**2.使用場合** <br>
對於一群不明特質的群體進行分類，且希望知道群的形成順序和關係演進，奇怪的資料分佈也可用<br>
a. 基因分群<br>

**3.限制** <br>
a. 距離計算和比較cluster方法改變，結果差很多=>無結論，選用有insight的方法<br>

**4.前置作業** <br>
a. (必做)Normalization：數值變數要做<br>
b. (Optional)Gower Matrix：混合變數要做這個<br>
d. (Optional)Feature Selection：先評估一下要放入的feature或用其他演算法輔助<br>

In [ ]:
import gower
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

In [ ]:
# 算出來所有狀況連一連的數字，並畫圖，看一看決定想分幾群
# 數值變數先標準化 參考Kmeans區
dm = gower.gower_matrix(df) # 非混合變數本步驟不需要
Zd = linkage(dm)
dendrogram(Zd) 

In [ ]:
# 想好群數下labels
labels = fcluster(Zd, 3, criterion='maxclust')
df.insert(0, "Cluster", labels, True)

In [ ]:
df_dum_detail.groupby('Cluster')['is_cart'].count()
df_dum_detail.groupby(['Cluster', 'is_cart']).count()